# Australian Credit Card Assessment

## Data


In [ ]:
import time
import theano
import theano.tensor as T
import numpy as np
from theano.sandbox import cuda


theano.config.floatX = 'float32'
cuda.use('gpu')
theano.config.compute_test_value = 'off'

path_data = r'data\australian.dat'
data = np.genfromtxt(path_data, delimiter=' ')
n_classes = 2

data_input    = np.asarray(data[:, 0:-1], dtype=theano.config.floatX)
data_label    = np.zeros(shape=(len(data[:, -1:]), n_classes), dtype=theano.config.floatX)
classes_names = np.asarray(['clase 1', 'clase 2'], dtype='<U10')

for i, c in enumerate(data[:, -1:]):
    data_label[i, int(c)] = 1.0


## Training

In [ ]:
import sys
sys.path.insert(0, r'../../')

from libML.nnet import MLP
from libML.trainers import TrainerMLP
from sklearn.cross_validation import ShuffleSplit
from utils.ClassifierMetrics import ClassifierMetrics

classifier = MLP(data_input.shape[1], [7], n_classes,
                 output_activation=T.tanh,
                 hidden_activation=[T.tanh], type_output="classifier")

trainerMLP = TrainerMLP(classifier, cost="MCC", lr_adapt="ADAGRAD",
                        initial_learning_rate=0.01, initial_momentum_rate=0.9, regularizer="L2+L1")

folds = 3
sss = ShuffleSplit(data_input.shape[0], n_iter=folds, test_size=None, train_size=0.7, random_state=0)
max_epoch = 200
validation_jump = 5

# Initialize metrics
metrics = ClassifierMetrics(classes_names)

for i, (train_set, test_set) in enumerate(sss):
    # data train and test
    data_train = data_input[train_set]
    data_test = data_input[test_set]
    label_train = data_label[train_set]
    label_test = data_label[test_set]

    # training
    tic = time.time()
    train_cost, test_cost, best_iteration, best_test_cost, best_test_output = \
        trainerMLP.trainer(data_train, label_train, data_test, label_test,
                           max_epoch=max_epoch, reg_L1=1e-2, reg_L2=1e-3, batch_size=32,
                           validation_jump=validation_jump, early_stop_th=4)
    toc = time.time()

    # Reset parameters
    classifier.reset()
    
    # Compute metrics
    predict_test_output = T.argmax(data_label[test_set], axis=1).eval()
    metrics.append_pred(classes_names[predict_test_output], classes_names[best_test_output])
    metrics.append_cost(train_cost, test_cost)
    
    print("%d Elapsed time [s]: %f" % (i, toc - tic))
    
print("FINISHED!")


## Results

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

metrics.print()
metrics.plot_confusion_matrix()
metrics.plot_cost(max_epoch)

plt.tight_layout()